In [1]:
#Namen der Inseln

Inseln = [
    "Alexandra_Bank",
    "Alison_Reef",
    "Amboyna_Cay",
    "Antelope_Reef",
    "Ardasier_Reef",
    "Barque_Canada_Reef",
    "Bombay_Reef",
    "Central_Reef",
    "Collins_Reef",
    "Commodore_Reef",
    "Cornwallis_South_Reef",
    "Cuarteron_Reef",
    "Discovery_Great_Reef",
    "Drummond_Island",
    "Duncan_Islands",
    "East_Reef",
    "Erica_Reef",
    "Fiery_Cross_Reef",
    "Flat_Island",
    "Gaven_Reefs",
    "Grainger_Bank",
    "Grierson_Reef",
    "Hughes_Reef",
    "Investigator_Shoal",
    "Itu_Aba_Island",
    "Johnson_Reef",
    "Ladd_Reef",
    "Lansdowne_Reef",
    "Lincoln_Islands",
    "Loaita_Cay",
    "Loaita_Island",
    "Mariveles_Reef",
    "Middle_Island",
    "Mischief_Reef",
    "Money_Island",
    "Namyit_Island",
    "Nanshan_Island",
    "North_Island",
    "North_Reef",
    "Northeast_Cay",
    "Observation_Bank",
    "Pattle_Island",
    "Pearson_Reef",
    "Petley_Reef",
    "Prince_Consort_Bank",
    "Prince_of_Wales_Bank",
    "Quanfu_Island",
    "Rifleman_Bank",
    "Robert_Island",
    "Sand_Cay",
    "Scarborough_Shoal",
    "Second_Thomas_Shoal",
    "Sin_Cowe_Island",
    "South_Island",
    "South_Reef",
    "South_Sand",
    "Southwest_Cay",
    "Spratly_Island",
    "Subi_Reef",
    "Swallow_Reef",
    "Tennent_Reef",
    "Thitu_Island",
    "Tree_Island",
    "Triton_Island",
    "Vanguard_Bank",
    "West_Reef",
    "West_Sand",
    "West_York_Island",
    "Woody_Island",
    "Yagong_Island"
]

In [2]:
import os
import numpy as np
import rasterio
import pandas as pd #für .csv-Tabelle


#Funktion zur Berechnung des NDWI erstellen
def compute_ndwi(tiff_path, green_band=1, nir_band=2):
    with rasterio.open(tiff_path) as src:
        green = src.read(green_band).astype(np.float32)
        nir = src.read(nir_band).astype(np.float32)

        pixel_size = src.res[0]
        pixel_area_m2 = pixel_size ** 2

    with np.errstate(divide='ignore', invalid='ignore'):
        ndwi = (green - nir) / (green + nir)
    ndwi[np.isinf(ndwi)] = np.nan

    return ndwi, pixel_area_m2

#Funktion, um die jeweiligen vorher/nachher-TIFFs ins Script zu laden
def get_images_for_insel(insel):
    base = r"C:\Users\Windows\Desktop\Master\GIS1\TIFFs"
    s2_1 = os.path.join(base, "S2_1", f"S2_1_{insel}.tif")
    s2_2 = os.path.join(base, "S2_2", f"S2_2_{insel}.tif")

    if not os.path.exists(s2_1):
        raise FileNotFoundError(s2_1)
    if not os.path.exists(s2_2):
        raise FileNotFoundError(s2_2)

    ndwi_1, pixel_area = compute_ndwi(s2_1)
    ndwi_2, _ = compute_ndwi(s2_2)

    return ndwi_1, ndwi_2, pixel_area


In [3]:
results = []

#For-Schleife, die für alle Inseln anzeigt, wie viele Pixel Land waren/sind (in m²):
for insel in Inseln:
    ndwi_1, ndwi_2, pixel_area = get_images_for_insel(insel)

    land_1 = np.where(np.isnan(ndwi_1), 0, ndwi_1 <= -0.05).astype(np.uint8) #-0.05 als Schwellenwert für Land, um Wolken mit ungefähr 0 rauszufiltern
    land_2 = np.where(np.isnan(ndwi_2), 0, ndwi_2 <= -0.05).astype(np.uint8)

    land_pixels_1 = np.sum(land_1)
    land_pixels_2 = np.sum(land_2)

    land_area_1 = land_pixels_1 * pixel_area
    land_area_2 = land_pixels_2 * pixel_area

    print(f"{insel}: Landfläche vorher (S2_1) = {land_area_1:.1f} m², nachher (S2_2) = {land_area_2:.1f} m²")

    results.append({
        "insel": insel,
        "land_1": land_1,
        "land_2": land_2,
        "land_area_1_m2": land_area_1,
        "land_area_2_m2": land_area_2
    })


Alexandra_Bank: Landfläche vorher (S2_1) = 0.0 m², nachher (S2_2) = 23200.0 m²
Alison_Reef: Landfläche vorher (S2_1) = 8700.0 m², nachher (S2_2) = 42200.0 m²
Amboyna_Cay: Landfläche vorher (S2_1) = 7900.0 m², nachher (S2_2) = 22700.0 m²
Antelope_Reef: Landfläche vorher (S2_1) = 167100.0 m², nachher (S2_2) = 35100.0 m²
Ardasier_Reef: Landfläche vorher (S2_1) = 300.0 m², nachher (S2_2) = 5000.0 m²
Barque_Canada_Reef: Landfläche vorher (S2_1) = 467300.0 m², nachher (S2_2) = 85900.0 m²
Bombay_Reef: Landfläche vorher (S2_1) = 451700.0 m², nachher (S2_2) = 3200.0 m²
Central_Reef: Landfläche vorher (S2_1) = 1406000.0 m², nachher (S2_2) = 220900.0 m²
Collins_Reef: Landfläche vorher (S2_1) = 159700.0 m², nachher (S2_2) = 0.0 m²
Commodore_Reef: Landfläche vorher (S2_1) = 100.0 m², nachher (S2_2) = 4700.0 m²
Cornwallis_South_Reef: Landfläche vorher (S2_1) = 7300.0 m², nachher (S2_2) = 114300.0 m²
Cuarteron_Reef: Landfläche vorher (S2_1) = 147600.0 m², nachher (S2_2) = 233900.0 m²
Discovery_Great_

In [4]:
#For-Schleife, die final ausrechnet wie viele m² dazugewonnen wurden:
for island_result in results:
    insel = island_result["insel"]
    area_1 = island_result["land_area_1_m2"]
    area_2 = island_result["land_area_2_m2"]

    diff = area_2 - area_1
    
    if diff > 0:
        status = "Landgewinn"
    elif diff < 0:
        status = "Verlust (Wolken?)"
    else:
        status = "unverändert"

    print(
        f"{insel}: {status} "
        f"({diff:+.0f} m²)" #.0 = keine Nachkommastellen
    )

Alexandra_Bank: Landgewinn (+23200 m²)
Alison_Reef: Landgewinn (+33500 m²)
Amboyna_Cay: Landgewinn (+14800 m²)
Antelope_Reef: Verlust (Wolken?) (-132000 m²)
Ardasier_Reef: Landgewinn (+4700 m²)
Barque_Canada_Reef: Verlust (Wolken?) (-381400 m²)
Bombay_Reef: Verlust (Wolken?) (-448500 m²)
Central_Reef: Verlust (Wolken?) (-1185100 m²)
Collins_Reef: Verlust (Wolken?) (-159700 m²)
Commodore_Reef: Landgewinn (+4600 m²)
Cornwallis_South_Reef: Landgewinn (+107000 m²)
Cuarteron_Reef: Landgewinn (+86300 m²)
Discovery_Great_Reef: Landgewinn (+4100 m²)
Drummond_Island: Verlust (Wolken?) (-159100 m²)
Duncan_Islands: Landgewinn (+38500 m²)
East_Reef: Landgewinn (+29300 m²)
Erica_Reef: Verlust (Wolken?) (-6000 m²)
Fiery_Cross_Reef: Landgewinn (+2732900 m²)
Flat_Island: Verlust (Wolken?) (-289300 m²)
Gaven_Reefs: Landgewinn (+10800 m²)
Grainger_Bank: Verlust (Wolken?) (-7600 m²)
Grierson_Reef: Landgewinn (+75300 m²)
Hughes_Reef: Landgewinn (+15300 m²)
Investigator_Shoal: Verlust (Wolken?) (-3481300 m

In [5]:
#.csv-Tabelle mit Ergebnissen erstellen lassen

rows = []

for result in results:
    land_2018 = result["land_area_1_m2"]
    land_2025 = result["land_area_2_m2"]
    diff = land_2025 - land_2018

    rows.append({
        "Name": result["insel"],
        "Landfläche 2018 [m²]": land_2018,
        "Landfläche 2025 [m²]": land_2025,
        "Differenz der Landflächen [m²]": diff
    })

df = pd.DataFrame(rows)

csv_path = r"C:\Users\Windows\Desktop\Master\GIS1/Landgewinn_Daten.csv"
df.to_csv(csv_path, index=False, encoding="utf-8-sig") #utf-8-sig kümmert sich um Sonderzeichen wie ä und ²

print(f"Tabelle gespeichert unter: {csv_path}")

Tabelle gespeichert unter: C:\Users\Windows\Desktop\Master\GIS1/Landgewinn_Daten.csv
